In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent,GroupChat,GroupChatManager
from autogen.coding import DockerCommandLineCodeExecutor
import json
from autogen.agentchat.contrib.capabilities import transform_messages, transforms

In [ ]:
# give USER model name, opneai api key and base url here
MODEL_NAME_USER = "gpt-3.5-turbo"
OPENAI_API_KEY = "sk-eb7c7b09A8"
OPENAI_BASE_URL_USER = "https://api/v1"
config_list_user = [{"model": MODEL_NAME_USER,"api_key": OPENAI_API_KEY,"base_url": OPENAI_BASE_URL_USER}]

# give EXAMINEE model name, opneai api key and base url here
OPENAI_BASE_URL_EXAMINEE = "http://127.0.0.1:8000/v1"
MODEL_NAME_EXAMINEE = "RankingGPT-bloom-7b"
config_list_examinee = [{"model": MODEL_NAME_EXAMINEE,"api_key": OPENAI_API_KEY,"base_url": OPENAI_BASE_URL_EXAMINEE}]

# OPENAI_BASE_URL_EXAMINEE = "https://api/v1"
# MODEL_NAME_EXAMINEE = "gpt-3.5-turbo"
# config_list_examinee = [{"model": MODEL_NAME_EXAMINEE,"api_key": OPENAI_API_KEY,"base_url": OPENAI_BASE_URL_EXAMINEE}]

In [ ]:
def createChat(system_message_user,system_message_examinee,user_input_first):
    # create an AssistantAgent instance named "user_proxy" with the LLM configuration to be a customer
    user_proxy = UserProxyAgent(
        name="customer",
        human_input_mode="NEVER",
        system_message=system_message_user,
        llm_config={
            "cache_seed": 32,  # seed 用户缓存中间结果
            "config_list": config_list_user, 
            "temperature": 0,
        },  
        max_consecutive_auto_reply=10,
        is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
        code_execution_config={
            "work_dir": "coding",
            "use_docker": False,  
        },
    )

    # create an AssistantAgent instance named "assistant_examinee" with the LLM configuration to be a saler
    assistant_examinee = AssistantAgent(
        name="examinee",
        system_message=system_message_examinee,
        is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
        llm_config={
            "cache_seed": 42,  # seed 用户缓存中间结果
            "config_list": config_list_examinee, 
            "temperature": 0,
        },  
    )

    context_handling = transform_messages.TransformMessages(
        transforms=[
            transforms.MessageHistoryLimiter(max_messages=18),
            transforms.MessageTokenLimiter(max_tokens=100, max_tokens_per_message=80, min_tokens=50),
        ]
    )
    context_handling.add_to_agent(user_proxy)
    context_handling.add_to_agent(assistant_examinee)

    # the assistant receives a message from the user, which contains the task description
    chatresult = user_proxy.initiate_chat(
        assistant_examinee,
        message=user_input_first,
        max_turns=7,
        silent = True
    )
    return chatresult
    
# save chat history 
def save_chat(chatresult, txt_path , scenario_eng="NONE"):
    chat_list = []
    for chat in chatresult.chat_history:
        chatter = 'assistant' if chat["role"]=='user' else 'user'
        content = chat["content"].replace("\n", " ")
        line = f"{chatter}: {content}"
        chat_list.append(line)
    with open (txt_path, "w") as f:
        f.write("scenario: "+ scenario_eng + "\n" + "\n".join(chat_list))

In [ ]:
def chat_start(model_name,scene,folder_path_chat_history,folder_path_chat_scenes):
    with open(f'{folder_path_chat_scenes}/{scene}.json', 'r') as f:
        test_samples = json.load(f)
    # test_samples = test_samples["scenarios"]
    for i, sample in enumerate(test_samples):
        # if i >= 5:
        #     break
        # if i < 5:
            # continue
        txt_path = f"{folder_path_chat_history}/chat_{model_name}_{scene}_{i}.txt"
        if not os.path.exists(txt_path):
            try:
                scenario_eng = sample['scenario_eng']
                system_message_user = sample['system_message_user']
                system_message_examinee = sample['system_message_examinee']
                user_input_first = sample['user_input_first']
                chatresult = createChat(system_message_user,system_message_examinee,user_input_first)
                save_chat(chatresult,txt_path,scenario_eng)
                print(f"************{model_name}_{scene}_{i}******SUCCESSED******")
            except Exception as e:
                print(f"************{model_name}_{scene}_{i}******FAILED******")
                print("发生异常：", str(e))
                continue

In [ ]:
scenes = ["saler","contacting","relationship","streamer","business"]
model_name = "5"
folder_path_chat_scenes = "./appeal_eval/chat_scenes"
folder_path_chat_history = f"./appeal_eval/chat_history/chat_autogen_based/model_{model_name}"

if not os.path.exists(folder_path_chat_history):
    os.makedirs(folder_path_chat_history)

for scene in scenes:
    print(f"************{model_name}_{scene}************")
    chat_start(model_name,scene,folder_path_chat_history,folder_path_chat_scenes)